# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [15]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [56]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np
document = ET.parse( 'C://Users/Cody/Downloads/data_wrangling_xml/data_wrangling_xml/data/mondial_database.xml' )
# print names of all countries with their infant mortality rate
# change from printing the elements to saving them in a dataframe
# update 7/3/2016: no longer prints all values, instead appends to two lists below
country = []
imr = []
for child in document.getroot():
    country.append(child.find('name').text)
    if(child.find('infant_mortality') == None) :
        imr.append(np.nan)
    else : imr.append(child.find('infant_mortality').text)

#The resulting arrays country and imr are entirely Strings so I need to recast imr as a float to sort.        
#This can only be done one element at a time
for i in range(len(imr)) :
    imr[i] = float(imr[i])

#This creates our output table, sorts the IMRs and then outputs the 10 lowest real values
output = pd.DataFrame([country,imr]).T
output.columns = ["Country","Infant Mortality Rate"]
output = output.sort_values("Infant Mortality Rate").head(10)
ind = [1,2,3,4,5,6,7,8,9,10]
pd.DataFrame(output.values, index=ind, columns = ["Country","Infant Mortality Rate"])
#Definitely used some inefficient code to get here, but at least it finally looks nice! 
#Answer to number 1.

,Country,Infant Mortality Rate
1,Monaco,1.81
2,Japan,2.13
3,Bermuda,2.48
4,Norway,2.48
5,Singapore,2.53
6,Sweden,2.6
7,Czech Republic,2.63
8,Hong Kong,2.73
9,Macao,3.13
10,Iceland,3.15


In [149]:
#2. Find the 10 cities with the largest popoulations
# I need to iterate through all of the country and city branches, pulling out country name, city name, and most recent population
# Therefore I will need to append three lists, repeating country and pulling out single city and population values
# Afterwards, I need to cast all population values as an int/float to sort
# Then we will be able to create the Table we need to output
#
country = []
city = []
pop = []

for element in document.iterfind('country'):
    #need to do all list assignments in second loop, element for country, subelement for others
    #element.getiterator() returns a list of elements, which can be individually accessed using element.find()
    for subelement in element.getiterator('city'):
        
        for subsub in subelement.getiterator('population') :
            #country
            country.append(element.find('name').text)
            #capital
            city.append(subelement.find('name').text)
            #population
            pop.append(subsub.items()) 
        
        #pop.append(type(p.find('population')))
out = pd.DataFrame((pd.DataFrame([country,city,pop]).T).values,columns = ["Country","City","City Elements"]).head(10)
out

,Country,City,City Elements
0,Albania,Tirana,"[(year, 1987)]"
1,Albania,Tirana,"[(year, 1990), (measured, estimate)]"
2,Albania,Tirana,"[(year, 2011), (measured, census)]"
3,Albania,Shkodër,"[(year, 1987)]"
4,Albania,Shkodër,"[(year, 2011), (measured, census)]"
5,Albania,Durrës,"[(year, 1987)]"
6,Albania,Durrës,"[(year, 2011), (measured, census)]"
7,Albania,Vlorë,"[(year, 1987)]"
8,Albania,Vlorë,"[(year, 2011), (measured, census)]"
9,Albania,Elbasan,"[(year, 1987)]"


In [107]:
root = document.getroot()
#root[0][0].text
#root[0][0].text  root[n][0] is country name
#root[1].find('gdp_total').text
type(element)

xml.etree.ElementTree.Element

NameError: name 'country' is not defined